In [40]:
import numpy as np

def correlated_path_generator_3d(S0_A, S0_B, S0_C, T, r, q_A, q_B, q_C, sigma_A, sigma_B, sigma_C, rho, N, Ni):
    """
    Generates Ni random paths for two correlated underlying asset prices S_t1 and S_t2 using a log-normal process.

    Parameters:
    S0_A, S0_B, S0_C : float
        Initial stock prices
    T : float
        Time to maturity (in years)
    r : float
        Risk-free interest rate (annual)
    q_A, q_B, q_C : float
        Dividend yields (annual) for each stock
    sigma_A, sigma_B, sigma_C : float
        Volatilities of the underlying assets
    rho : float
        Correlation coefficient between the two assets
    N : int
        Number of time steps
    Ni : int
        Number of paths to generate

    Returns:
    tuple of numpy.ndarray
        Arrays of shape (Ni, N+1) containing Ni random paths for S_tA, S_tB and S_tC
    """
    dt = T / N
    paths_A = np.zeros((Ni, N+1))
    paths_B = np.zeros((Ni, N+1))
    paths_C = np.zeros((Ni, N+1))

    paths_A[:, 0] = S0_A
    paths_B[:, 0] = S0_B
    paths_C[:, 0] = S0_C
    
    # Cholesky decomposition to generate correlated random variables
    A = np.linalg.cholesky(corr_matrix)
    
    for i in range(1, N+1):
        Z = np.random.normal(0, 1, (Ni, 3))
        Z_corr = Z.dot(A.T)  # Apply the Cholesky matrix to get correlated random variables
        paths_A[:, i] = paths_A[:, i-1] * np.exp((r - q_A - 0.5 * sigma_A**2) * dt + np.sqrt(dt) * sigma_A * Z_corr[:, 0])
        paths_B[:, i] = paths_B[:, i-1] * np.exp((r - q_B - 0.5 * sigma_B**2) * dt + np.sqrt(dt) * sigma_B * Z_corr[:, 0])
        paths_C[:, i] = paths_C[:, i-1] * np.exp((r - q_C - 0.5 * sigma_C**2) * dt + np.sqrt(dt) * sigma_C * Z_corr[:, 0])

    
    return paths_A, paths_B, paths_C

# Parameters based on pdf
S0_A, S0_B, S0_C = 150, 250, 300  # Initial stock prices
T = 5  # Time to maturity in years
r = 0.02  # Risk-free interest rate
q_A, q_B, q_C = 0.015, 0.02, 0.025  # Dividend yields
sigma_A, sigma_B, sigma_C = 0.2, 0.22, 0.25  # Volatilities
N = 365 * T  # Number of time steps (daily steps)
Ni = 10000  # Number of paths

# Test the function with example parameters
paths_A, paths_B, paths_C = correlated_path_generator_3d(S0_A, S0_B, S0_C, T, r, q_A, q_B, q_C, sigma_A, sigma_B, sigma_C, corr_matrix, N, Ni)

# Let's check the shape of the generated paths to confirm everything is working as expected
paths_A.shape, paths_B.shape, paths_C.shape

In [54]:
A = np.linalg.cholesky([[1, 0.8], [0.8, 1]]) #Check
A.dot(A.T)
# A

array([[1. , 0.8],
       [0.8, 1. ]])

In [58]:
Z = np.random.normal(0, 1, (1000000, 2))
Z_corr = Z.dot(A.T)

np.array([[np.mean(Z_corr[:, i] * Z_corr[:, j]) for i in range(2)] for j in range(2)])

array([[1.00149513, 0.80081436],
       [0.80081436, 1.00010868]])

In [43]:
def best_of_relative_change_payoff(S_A, S_B, S_C, SA_0, SB_0, SC_0, Notional, kappa):
    """
    Calculates the payoff of a note based on the "best of" relative performance of two assets,
    adjusted by a relative strike (kappa).

    Parameters:
    SA, SB, SC : numpy.ndarray
        Final stock prices for each path
    SA_0, SB_0, SC_0 : float
        Initial stock prices
    Notional : float
        Notional value representing the dimension of money
    kappa : float
        Relative strike

    Returns:
    numpy.ndarray
        Payoff for each path
    """
    relative_performance_A = SA / SA_0
    relative_performance1_B = SB / SB_0
    relative_performance_C = SC / SC_0

    best_of_relative_performance = np.maximum(relative_performance_A, relative_performance_B, relative_performance_C)
    payoff = Notional * np.maximum(best_of_relative_performance - kappa, 0)
    
    return payoff

In [44]:
def price_note_with_relative_change(pathsA, pathsB, paths_C, S0_A, S0_B, S0_C, Notional, kappa, r, T):
    """
    Calculates the price of a note based on the "best of" relative performance of two assets,
    adjusted by a relative strike (kappa), and discounts it to present value.

    Parameters:
    paths1, paths2 : numpy.ndarray
        Paths of stock prices for each asset
    SA_0, SB_0, SC_0 : float
        Initial stock prices for each asset
    Notional : float
        Notional value representing the dimension of money
    kappa : float
        Relative strike
    r : float
        Risk-free interest rate (annual)
    T : float
        Time to maturity (in years)

    Returns:
    tuple
        Mean price of the note and its standard error
    """
    # Calculate the payoff at maturity for each path using the modified payoff function
    payoffs = best_of_relative_change_payoff(pathsA[:, -1], pathsB[:, -1], paths[:, -1] S0_A, S0_B, S0_C, Notional, kappa)
    
    # Discount payoffs to present value
    pv = np.exp(-r * T) * payoffs
    
    # Calculate mean price and standard error
    mean_price = np.mean(pv)
    std_error = np.std(pv) / np.sqrt(len(pv))
    
    return mean_price, std_error

In [62]:
# Parameters based on the document
S0_A = 150
S0_B = 250
S0_C = 300
T = 5 # Time to maturity in years
r = 0.02 # Risk-free interest
# Volatilities
q_A = 0.015
q_B = 0.02
q_C = 0.025
# Dividend yields
sigma_A = 0.2
sigma_B = 0.22
sigma_C = 0.25
corr_matrix = np.array([[1, 0.75, 0.85], [0.75, 1, 0.9], [0.85, 0.9, 1]])  # Correlation matrix
# Check
rho = 0.999
N = 20
Ni = 10000
Notional = 100000
kappa = 1.05
#

# Generate correlated paths
paths_A, paths_B, paths_C = correlated_path_generator3d(S0_A, S0_B, S0_C, T, r, q_A, q_B, q_C, sigma_A, sigma_B, sigma_C, rho, N, Ni)

# Price the note
mean_price, std_error = price_note_with_relative_change(paths_A, paths_B, paths_C, S0_A, S0_B, S0_C, Notional, kappa, r, T)

print(f"Mean Price of the Note: ${mean_price:.2f}")
print(f"Standard Error: ${std_error:.2f}")

Mean Price of the Note: $87.12
Standard Error: $0.16


In [ ]:
# Implementing the logic for the autocall feature, final payoff at maturity, and a framework for calculating the Greeks

def calculate_payoff_and_greeks(paths_A, paths_B, paths_C, initial_prices, autocall_dates, T, r, notional, coupon_rate, final_levels):
    """
    Calculates the payoff based on the autocall feature and the final payoff at maturity.
    Also outlines the approach for calculating the Greeks.
    
    Parameters:
    paths_A, paths_B, paths_C : numpy.ndarray
        Simulated paths for the three stocks
    initial_prices : tuple
        Initial stock prices for A, B, and C
    autocall_dates : list
        Indices of autocall observation dates in the paths
    T : float
        Time to maturity (in years)
    r : float
        Risk-free interest rate
    notional : float
        Notional amount of the derivative
    coupon_rate : float
        Annualized coupon rate
    final_levels : dict
        Thresholds for calculating the final payoff
    
    Returns:
    float
        Expected payoff of the derivative
    """
    # Placeholder for expected payoff calculation
    expected_payoff = 0.0
    
    # Placeholder for Greeks calculation
    delta, gamma, vega, interest_rate_delta = 0.0, 0.0, 0.0, 0.0
    
    # Logic for calculating the payoff and Greeks to be implemented here
    
    return expected_payoff, (delta, gamma, vega, interest_rate_delta)

# Example parameters
initial_prices = (150, 250, 300)  # Initial stock prices for A, B, and C
autocall_dates = [int(365 * (1/2 + i)) for i in range(T)]  # Semi-annual autocall observation dates
notional = 1000000  # Notional amount
coupon_rate = 0.07  # Annualized coupon rate
final_levels = {'upper': 1.15, 'lower': 0.60}  # Thresholds for final payoff calculation

# Call the function with example parameters to calculate the expected payoff and outline the Greeks calculation
payoff, greeks = calculate_payoff_and_greeks(paths_A, paths_B, paths_C, initial_prices, autocall_dates, T, r, notional, coupon_rate, final_levels)

# Displaying the expected payoff and placeholder values for the Greeks as we've not implemented the logic yet
 return payoff, greeks

In [ ]:
import numpy as np
from scipy.stats import norm

# Parameters from pdf
initial_stock_prices = np.array([150, 250, 300])  # Stock A, B, C
auto_call_trigger_level = 1.15  # 115%
notional_amount = 1_000_000  # Notional Amount
coupon_rate = 0.07  # Annual coupon rate
maturity_coupon = 0.15  # Coupon if stock is above 115% at maturity
maturity_years = 5  # 5 years to maturity
observation_dates_years = np.array([1, 2, 3, 4, 5])  # Annual observations for simplicity

# Volatility, interest rates, and dividend yields as constant averages for simplicity
volatilities = np.array([0.20, 0.22, 0.25])
interest_rates = 0.02
dividend_yields = np.array([0.015, 0.02, 0.025])

# Correlation matrix
correlation_matrix = np.array([
    [1, 0.75, 0.85],
    [0.75, 1, 0.9],
    [0.85, 0.9, 1]
])

# Simulation parameters
num_simulations = 10_000  # Adjust as needed
time_steps = int(maturity_years * 252)  # Daily steps for each year

# For Greeks calculation
delta_shift = 0.01  # 1% shift for Delta and Gamma
vega_shift = 0.01  # 1% shift in volatility for Vega
rho_shift = 0.0001  # 0.01% shift in interest rate for Rho

def simulate_paths(S0, T, r, q, sigma, corr_matrix, steps, simulations):
    # Generate paths for each stock using GBM and Cholesky decomposition for correlation
    pass  # Complete this function based on the given parameters and stock path generation logic

def calculate_payoff(paths, notional, trigger_level, coupon_rate, maturity_coupon, observation_dates, maturity):
    # Calculate the payoff for the note based on the paths and the note's terms
    pass  # Complete this function to calculate payoffs based on autocall logic and final payoff conditions

def estimate_greeks(paths, S0, vol, r, q, notional, trigger_level, coupon_rate, maturity_coupon, observation_dates, maturity):
    # Calculate Delta, Gamma, Vega, and Rho based on perturbed parameters and revaluation of the derivative
    pass  # Complete this function to estimate the Greeks

# Main simulation
stock_paths = simulate_paths(initial_stock_prices, maturity_years, interest_rates, dividend_yields, volatilities, correlation_matrix, time_steps, num_simulations)

# Calculate the payoffs from the simulation
payoffs = calculate_payoff(stock_paths, notional_amount, auto_call_trigger_level, coupon_rate, maturity_coupon, observation_dates_years, maturity_years)

# Discount the payoffs to present value and estimate price
discounted_payoffs = payoffs * np.exp(-interest_rates * maturity_years)
price_estimate = np.mean(discounted_payoffs)

# Estimate the Greeks
greek_estimates = estimate_greeks(stock_paths, initial_stock_prices, volatilities, interest_rates, dividend_yields, notional_amount, auto_call_trigger_level, coupon_rate, maturity_coupon, observation_dates_years, maturity_years)

# Output the estimated price and Greeks
print(f"Price Estimate: {price_estimate}")
print(f"Greeks: {greek_estimates}")

